In [ ]:

import duckdb
con_path = "C:\lopu-kg-test\project\initial_db.duckdb"
con = duckdb.connect(con_path)
con.sql("CALL start_ui();")
con.close()

In [ ]:
import duckdb

# Connect to your DB
con_path = "C:\lopu-kg-test\project\initial_db.duckdb"
con = duckdb.connect(con_path)
# Get table list
tables = con.execute("""
    SELECT table_schema, table_name
    FROM information_schema.tables
    WHERE table_catalog = 'initial_db'
""").fetchall()

# Now query each table dynamically
results = []
for schema, table in tables:
    count = con.execute(f"SELECT COUNT(*) FROM {schema}.{table}").fetchone()[0]
    results.append((schema, table, count))

con.close()
# Output
for row in results:
    if row[2] == 0:
        print(row)

IOException: IO Error: File is already open in 
C:\Users\kasutaja\AppData\Local\Programs\Python\Python311\python.exe (PID 16344)

In [9]:
# siin pooleli
import os
import re

import duckdb

# Connect to your DB
#con_path = "C:\lopu-kg-test\project\initial_db.duckdb"
con = duckdb.connect()


def find_finwire_files(folder_path):
    """
    Traverses a folder recursively and finds files matching the pattern
    FINWIRE<YYYY>Q<Q> (e.g., FINWIRE2016Q3, FINWIRE2016Q4.txt).

    It specifically looks for filenames where the part *before* the extension
    exactly matches the pattern 'FINWIRE' followed by 4 digits (year),
    'Q', and 1 digit (quarter). Files with additional suffixes before the
    extension (like '_audit' in FINWIRE1967Q2_audit.csv) are skipped.

    Args:
        folder_path (str): The path to the folder to traverse.

    Returns:
        list: A list of full paths to the matching files.
               Returns an empty list if the folder doesn't exist or no
               matching files are found.
    """
    matching_files = []

    # Regex pattern:
    # ^        - anchor to the start of the string
    # FINWIRE  - literal string
    # \d{4}    - exactly 4 digits (for the year)
    # Q        - literal character 'Q'
    # \d       - exactly 1 digit (for the quarter)
    # $        - anchor to the end of the string
    # This pattern ensures the base name matches exactly.
    pattern = re.compile(r"^FINWIRE\d{4}Q\d$")

    # Check if the folder path exists
    if not os.path.isdir(folder_path):
        print(f"Warning: Folder not found at {folder_path}")
        return matching_files

    # os.walk traverses the directory tree (root, dirs, files)
    for root, _, files in os.walk(folder_path):
        for filename in files:
            # Split the filename into the base name and the extension
            base_name, _ = os.path.splitext(filename)

            # Check if the base name matches the compiled regex pattern
            if pattern.match(base_name):
                # If it matches, construct the full path and add it to the list
                full_path = os.path.join(root, filename)
                matching_files.append(full_path)

    return matching_files

src_folder = 'C:\lopu-kg-test\project\src\data\Batch1'
find_finwire_files(src_folder)


list_of_finwire_files = find_finwire_files(src_folder)
# file_path = os.path.join(src_folder, "FINWIRE2016Q3")

asd = """
 

 CREATE TABLE IF NOT EXISTS FinWire (
   rectype STRING,
   recdate date,
   value STRING
 ) 
;
 

 """
con.sql(asd)

file_path = src_folder


for file_path in list_of_finwire_files:
    #file_path = os.path.join(src_folder, file)
    # õige
    fin_wire_sql = f""" 
    INSERT INTO FinWire
    SELECT
        CASE
            WHEN SUBSTR(column0, 16, 3) = 'FIN' THEN
                CASE
                    WHEN TRY_CAST(TRIM(SUBSTR(column0, 187, 60)) AS BIGINT) IS NOT NULL THEN 'FIN_COMPANYID'
                    ELSE 'FIN_NAME'
                END
            ELSE SUBSTR(column0, 16, 3)
        END AS rectype,
        STRPTIME(SUBSTR(column0, 1, 8), '%Y%m%d') AS recdate,
        SUBSTR(column0, 19) AS value
    FROM read_csv_auto('{file_path}', HEADER=FALSE, filename=false, all_varchar=true)
    """
    con.sql(fin_wire_sql)
    #save_sql_to_file(fin_wire_sql, "wh_db_stage.FinWire")
    print(file_path, "--DONE--")


C:\lopu-kg-test\project\src\data\Batch1\FINWIRE1967Q1 --DONE--
C:\lopu-kg-test\project\src\data\Batch1\FINWIRE1967Q2 --DONE--
C:\lopu-kg-test\project\src\data\Batch1\FINWIRE1967Q3 --DONE--
C:\lopu-kg-test\project\src\data\Batch1\FINWIRE1967Q4 --DONE--
C:\lopu-kg-test\project\src\data\Batch1\FINWIRE1968Q1 --DONE--
C:\lopu-kg-test\project\src\data\Batch1\FINWIRE1968Q2 --DONE--
C:\lopu-kg-test\project\src\data\Batch1\FINWIRE1968Q3 --DONE--
C:\lopu-kg-test\project\src\data\Batch1\FINWIRE1968Q4 --DONE--
C:\lopu-kg-test\project\src\data\Batch1\FINWIRE1969Q1 --DONE--
C:\lopu-kg-test\project\src\data\Batch1\FINWIRE1969Q2 --DONE--
C:\lopu-kg-test\project\src\data\Batch1\FINWIRE1969Q3 --DONE--
C:\lopu-kg-test\project\src\data\Batch1\FINWIRE1969Q4 --DONE--
C:\lopu-kg-test\project\src\data\Batch1\FINWIRE1970Q1 --DONE--
C:\lopu-kg-test\project\src\data\Batch1\FINWIRE1970Q2 --DONE--
C:\lopu-kg-test\project\src\data\Batch1\FINWIRE1970Q3 --DONE--
C:\lopu-kg-test\project\src\data\Batch1\FINWIRE1970Q4 -

In [10]:
con.sql("select distinct rectype from FinWire")

┌───────────────┐
│    rectype    │
│    varchar    │
├───────────────┤
│ SEC           │
│ FIN_COMPANYID │
│ FIN_NAME      │
│ CMP           │
└───────────────┘

In [2]:
import duckdb

# Connect to your DB
con_path = "C:\lopu-kg-test\project\initial_db.duckdb"
con = duckdb.connect(con_path)
con.sql("CALL start_ui();")
#con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────────────────────────────┐
│                result                │
│               varchar                │
├──────────────────────────────────────┤
│ UI started at http://localhost:4213/ │
└──────────────────────────────────────┘